# Qué hay acá?

En esta notebook vectorizo los tweets de los políticos.

Nota: Mi pc no tiene recursos para correr todo el dataset, así que voy a correrlo en Google Colab.

In [1]:
#!pip install pytorch-transformers==1.2.0 torch==1.10.1 torchaudio==0.10.1 torchtext==0.11.1 torchvision==0.11.2 transformers==4.13.0

In [2]:
import pandas as pd
import torch
from pytorch_transformers import *

from google.colab import drive
drive.mount('drive')

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


device(type='cuda')

In [3]:
politico = 'catacora'

#'drive/MyDrive/Colab-Notebooks/prueba-bert'
df = pd.read_excel('drive/MyDrive/Colab-Notebooks/tweets-politicos-hs/data/raw/' + politico + '-preproc.xlsx')

df = df["texto-original"]

# drop duplicates
df = df.drop_duplicates()

print(df.shape)
df.head()

(1931,)


0    En Casa Grande del Pueblo escuchamos las preoc...
1    Hablar de nuestra Central Obrera Departamental...
2    #ENVIVO\nEs un honor participar en el aniversa...
3    Saludamos el compromiso de las organizaciones ...
4    El municipio de Laja es una de las joyas que t...
Name: texto-original, dtype: object

In [4]:
# apply tokenizer to tweets in df
tokenized = df.apply((lambda x: torch.tensor([tokenizer.encode(x, add_special_tokens=True)])))

tokenized

0       [[tensor(101), tensor(10243), tensor(15076), t...
1       [[tensor(101), tensor(13740), tensor(22491), t...
2       [[tensor(101), tensor(108), tensor(31278), ten...
3       [[tensor(101), tensor(85198), tensor(73983), t...
4       [[tensor(101), tensor(10224), tensor(14059), t...
                              ...                        
1937    [[tensor(101), tensor(11045), tensor(55818), t...
1938    [[tensor(101), tensor(16570), tensor(108), ten...
1939    [[tensor(101), tensor(14563), tensor(92837), t...
1940    [[tensor(101), tensor(10224), tensor(75632), t...
1941    [[tensor(101), tensor(27158), tensor(26331), t...
Name: texto-original, Length: 1931, dtype: object

In [5]:
tokenized.shape[0]

1931

In [6]:
num = '20'
cont = (int(num)-1)*100

# apply model to tokenized
if cont < tokenized.shape[0]:
  last_hidden_states = tokenized.iloc[cont:cont+100].apply(lambda x: model(x)[0])
else:
  last_hidden_states = tokenized.iloc[cont:].apply(lambda x: model(x)[0])

# compute average
average = last_hidden_states.apply(lambda x: torch.mean(x, dim=1))

#average.head()

#average.to_json('drive/MyDrive/Colab-Notebooks/tweets-politicos-hs/data/intermediate/' + politico + '-vectores_1.json', orient='records', lines=True)

# convert to numpy
average = average.apply(lambda x: x.detach().numpy())
# dimension (1, 768) to (768,)
average = average.apply(lambda x: x[0])

average.to_json('drive/MyDrive/Colab-Notebooks/tweets-politicos-hs/data/intermediate/' + politico + '-vectores_' + num + '.json', orient='records')
torch.cuda.empty_cache()

average.head()

1911    [-0.6035465, -0.33645505, 0.16429396, -0.02722...
1912    [-0.34263492, -0.37672547, 0.08698719, 0.21320...
1913    [-0.086646356, -0.47567984, 0.23335615, 0.1654...
1914    [-0.23527135, -0.46947682, 0.3462993, 0.194951...
1915    [-0.33237976, -0.2958282, 0.29307863, 0.105809...
Name: texto-original, dtype: object